In [1]:
import os

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint

import os
import tqdm
import copy
import random
import logging
from absl import app
from absl import flags
from torch.utils.data import TensorDataset, DataLoader

import nam.metrics
import nam.data
from nam.model import *
from nam.pl import *

FLAGS = flags.FLAGS

In [2]:
config = defaults()
print(config)

Config(activation='exu', batch_size=1024, cross_val=False, data_path='data/GALLUP.csv', decay_rate=0.995, device='cpu', dropout=0.5, early_stopping_patience=50, experiment_name='NAM', feature_dropout=0.25, hidden_layer='relu', hidden_units=[64, 32, 32], id_fold=1, l2_regularization=0.001, learning_rate=0.00674, logdir='output', n_basis_functions=1024, n_folds=5, n_splits=3, num_epochs=10, num_models=1, output_regularization=0.01, regression=True, save_model_frequency=2, save_top_k=3, seed=2021, shallow_layer='exu', shuffle=True, units_multiplier=2, use_dnn=False, wandb=False)


In [3]:
nam_data = NAMDataModule(config)

In [4]:
nam_data.tmp_x_train

array([[-0.358533  , -0.21568628, -0.9350129 , ..., -0.99727434,
         0.01806589, -0.52589643],
       [-0.9338216 ,  0.21568628, -0.94332844, ..., -0.9968143 ,
         0.1498406 , -0.6055777 ],
       [-0.70525926,  0.29411766, -0.93020326, ..., -0.9953849 ,
         0.10095643, -0.37649402],
       ...,
       [-0.1416808 ,  1.        , -0.9066077 , ..., -0.99659395,
         0.10095643, -0.62350595],
       [-0.5517165 , -0.5686275 , -0.9386992 , ..., -0.998227  ,
        -0.7045696 ,  0.2808765 ],
       [-0.49677935, -0.05882353, -0.9314878 , ..., -0.9969146 ,
         0.1498406 , -0.5239044 ]], dtype=float32)

In [3]:
train, (x_test, y_test) = nam.data.create_test_train_fold(dataset='sklearn_housing',
                                                          id_fold=config.id_fold,
                                                          n_folds=config.n_folds,
                                                          n_splits=config.n_splits,
                                                          regression=not config.regression)
test_dataset = TensorDataset(torch.tensor(x_test), torch.tensor(y_test))
test_dl = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=not config.shuffle)

In [4]:
train_ds = list(train)

In [5]:
x_trains, y_trains, x_validates, y_validates = [],[],[],[]
for fold in train_ds:
    (x_train, y_train), (x_validate, y_validate) = fold
    x_trains.append(torch.tensor(x_train))
    y_trains.append(torch.tensor(y_train))
    x_validates.append(torch.tensor(x_validate))
    y_validates.append(torch.tensor(y_validate))

In [6]:
tmp_x_train = x_train

In [7]:
x_train = torch.cat(x_trains)
y_train = torch.cat(y_trains)
x_validate = torch.cat(x_validates)
y_validate = torch.cat(y_validates)

In [8]:
x_train.shape, x_validate.shape

(torch.Size([43344, 8]), torch.Size([6192, 8]))

In [9]:
# torch.cat(x_trains, dim=0).shape, x_train.shape

In [10]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=config.batch_size, shuffle=config.shuffle, num_workers=8)

val_ds = TensorDataset(x_validate, y_validate)
val_dl = DataLoader(val_ds, batch_size=config.batch_size, shuffle=not config.shuffle, num_workers=8)

In [11]:
# init model
nam_model = NAM(config=config, x_train=tmp_x_train)

# Initialize a trainer
checkpoint_callback = ModelCheckpoint(
    monitor="MAE_epoch",
    dirpath="/home/amrmkayid/nam/models/new/",
    filename="nam-{epoch:02d}-{MAE_epoch:.2f}",
    save_top_k=3,
    mode="min",
)
trainer = pl.Trainer(max_epochs=config.num_epochs, callbacks=[checkpoint_callback])  #, progress_bar_refresh_rate=20)

# # Train the model ⚡
trainer.fit(nam_model, train_dl, val_dl)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type                | Params
----------------------------------------------
0 | model | NeuralAdditiveModel | 505 K 
----------------------------------------------
505 K     Trainable params
0         Non-trainable params
505 K     Total params
2.022     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/amrmkayid/miniconda3/envs/nam/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning: The number of training samples (43) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [12]:
trainer.test(test_dataloaders=test_dl)

/home/amrmkayid/miniconda3/envs/nam/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:678: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  rank_zero_deprecation(
/home/amrmkayid/miniconda3/envs/nam/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_MAE': 0.8629046082496643, 'test_loss': 1.3837798833847046}
--------------------------------------------------------------------------------


[{'test_loss': 1.3837798833847046, 'test_MAE': 0.8629046082496643}]